In [1]:
%pylab inline

from astropy.io import fits
import os

from styles.colors import *
from styles.slides import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
idir = os.path.expandvars("$phd/inputs/real_deal/fits")

fits_list = sorted([os.path.join(root, file) for root, subs, files in os.walk(idir) for file in files if file.endswith(".fit")])

In [3]:
index_names = [
    "4000Abreak",
    "Lick_Hb",
    "Lick_Mg2",
    "Lick_Fe4531",
    "Lick_Fe5015",
    "Lick_Mgb",
    "Lick_Fe5270",
    "Lick_Fe5335"
]
pos = {
    "4000Abreak": (5, "ew", 34),
    "Lick_Hb": (5, "ew", 8),
    "Lick_Mg2": (5, "mag", 11),            #should be in mag
    "Lick_Fe4531": (5, "ew", 6),
    "Lick_Fe5015": (5, "ew", 9),
    "Lick_Mgb": (5, "mag", 12),            #should be in mag
    "Lick_Fe5270": (5, "ew", 13),
    "Lick_Fe5335": (5, "ew", 14),
    "Hd": (2, "ew", 24),
    "Hg": (2, "ew", 26),
    
    "4000Abreak_e": (5, "ewErr", 34),
    "Lick_Hb_e": (5, "ewErr", 8),
    "Lick_Mg2_e": (5, "magErr", 11),            #should be in mag
    "Lick_Fe4531_e": (5, "ewErr", 6),
    "Lick_Fe5015_e": (5, "ewErr", 9),
    "Lick_Mgb_e": (5, "magErr", 12),            #should be in mag
    "Lick_Fe5270_e": (5, "ewErr", 13),
    "Lick_Fe5335_e": (5, "ewErr", 14),
    "Hd_e": (2, "ewErr", 24),
    "Hg_e": (2, "ewErr", 26),
}
kws = ["D4000", "Hbeta", "Hdg", "Mg2Fe", "MgFep"]
g05_index_ingredients = {
    "D4000": ("4000Abreak",),
    "Hbeta": ("Lick_Hb",),
    "Hdg": ("Hd", "Hg"),
    "Mg2Fe": ("Lick_Mg2", "Lick_Fe4531", "Lick_Fe5015"),
    "MgFep": ("Lick_Mgb", "Lick_Fe5270", "Lick_Fe5335")
}
g05_index_ingredients_e = {
    "D4000": ("4000Abreak", "4000Abreak_e"),
    "Hbeta": ("Lick_Hb_e",),
    "Hdg": ("Hd_e", "Hg_e"),
    "Mg2Fe": ("Lick_Mg2", "Mg2Fe", "Lick_Mg2_e", "Lick_Fe4531_e", "Lick_Fe5015_e"),
    "MgFep": ("Lick_Mgb", "MgFep", "Lick_Mgb_e", "Lick_Fe5270_e", "Lick_Fe5335_e")
}
g05_index_directions = {
    "D4000": lambda D4000: 1./D4000,
    "Hbeta": lambda Hb: Hb,
    "Hdg": lambda Hd, Hg: Hd + Hg,
    "Mg2Fe": lambda Mg2, Fe4531, Fe5015: 0.6*Mg2 + 0.4*log10(abs(Fe4531+Fe5015)),
    "MgFep": lambda Mgb, Fe5270, Fe5335: sqrt(abs(Mgb*(0.72*Fe5270+0.28*Fe5335)))
}
g05_index_directions_e = {
    "D4000": lambda D4000, D4000_e: 1./D4000**2*D4000_e,
    "Hbeta": lambda Hb_e: Hb_e,
    "Hdg": lambda Hd_e, Hg_e: sqrt(Hd_e**2 + Hg_e**2),
    "Mg2Fe": lambda Mg2, Mg2Fe, Mg2_e, Fe4531_e, Fe5015_e: sqrt((0.6*Mg2_e)**2 + (0.4*Fe4531_e*10**(-Mg2Fe))**2 + (0.4*Fe5015_e*10**(-Mg2Fe))**2),
    "MgFep": lambda Mgb, MgFep, Mgb_e, Fe5270_e, Fe5335_e: sqrt((MgFep*0.5/Mgb*Mgb_e)**2 + (0.72*Mgb*0.5/MgFep*Fe5270_e)**2 + (0.28*Mgb*0.5/MgFep*Fe5335_e)**2)
}

In [4]:
g05_index = {
    "D4000": [],
    "Hbeta": [],
    "Hdg": [],
    "Mg2Fe": [],
    "MgFep": []
}
g05_index_errors = {
    "D4000": [],
    "Hbeta": [],
    "Hdg": [],
    "Mg2Fe": [],
    "MgFep": []
}
g05_table = []
for file in fits_list:
    f = fits.open(file)

    for field in kws:
        ingredients = ()
        for ingredient in g05_index_ingredients[field]:
            hdu, table, row = pos[ingredient]
            ingredients += (f[hdu].data[table][row],)
        g05_index[field] += [g05_index_directions[field](*ingredients)]
        ingredients_e = ()
        for ingredient in g05_index_ingredients_e[field]:
            if ingredient=="Mg2Fe":
                ingredients_e += (g05_index[ingredient][-1],)
            elif ingredient=="MgFep":
                ingredients_e += (g05_index[ingredient][-1],)
            else:
                hdu, table, row = pos[ingredient]
                ingredients_e += (f[hdu].data[table][row],)
        g05_index_errors[field] += [g05_index_directions_e[field](*ingredients_e)]

In [5]:
#These are the errors used in the G05 indexes plots
for index in g05_index_ingredients: print "{}: {}".format(index, percentile(g05_index_errors[index], 50.))

Hdg: 0.491263576004
Mg2Fe: 0.0980696651773
Hbeta: 0.314397871494
MgFep: 0.0453182593428
D4000: 0.0167891783384


In [6]:
fields = ["input_file", "D4000", "Hbeta", "Hdg", "Mg2Fe", "MgFep", "D4000_err", "Hbeta_err", "Hdg_err",
          "Mg2Fe_err", "MgFep_err"]
fmt = "%25s"+"%13.5f"*(len(fields)-1)

g05_table = zeros(len(fits_list), dtype=zip(fields, ["|S25"]+["<f8"]*(len(fields)-1)))
g05_table["input_file"] = map(lambda path: path.split("/")[-1], fits_list)
for field in fields[1:]:
    if field.endswith("_err"):
        g05_table[field] = g05_index_errors[field[:-4]]
    else:
        g05_table[field] = g05_index[field]

savetxt("data/RealDeal/G05_indexes_sample.txt", g05_table, fmt, header=("%23s"+"%13s"*(len(fields)-1))%tuple(fields))